### Imports and configuration

In [2]:
import boto3
import sagemaker
import sagemaker
from sagemaker.huggingface.processing import HuggingFaceProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
language = "en" #change this to de, making sure the data is in german
output_file_name = 'augmented_reviews.csv'

In [3]:
input_data = f's3://{bucket}/sm-data-aug-nlp/inputs/imdb.csv'

### Instantiate a HuggingFaceProcessor with Job configurations

In [ ]:
hf_processor = HuggingFaceProcessor(
    role = role, 
    instance_type = 'ml.p3.2xlarge',
    transformers_version = '4.6',
    pytorch_version = '1.7',
    instance_count = 1,
)

### Set data and arguments configuration
We decouple the choice of source language from the processing script; the `language` parameter can be either `de` or `en`.

In [ ]:
inputs = [ProcessingInput(
    source = input_data, 
    destination = "/opt/ml/processing/input")
         ]
outputs = [ProcessingOutput(
    output_name = 'augmented-text', 
    source="/opt/ml/processing/output", 
    destination =f's3://{bucket}/sm-data-aug-nlp/output/')
          ]
arguments = ["--file-name", "imdb.csv", 
             "--output-file-name", output_file_name,
             "--language", language]

### Start the Processing Job

In [ ]:
hf_processor.run(
    code = 'scripts/aug-hf.py',
    inputs = inputs, 
    outputs = outputs, 
    arguments = arguments,
    wait = False
)